In [1]:
import networkx as nx
import pandas as pd
import pulp
import numpy as np
import platform
import netwulf

In [2]:
G = nx.read_graphml("shareability_graph.graphml")

In [5]:
print(f"no. nodes: {len(list(G.nodes))} \n no. edges: {len(list(G.edges))}")

no. nodes: 198 
 no. edges: 748


In [ ]:
netwulf.visualize(G)

#### usuwamy wszystkie izolowane node'y

In [3]:
nodes_to_keep = [k for k, v in dict(G.degree).items() if v!=0]

In [4]:
g = G.subgraph(nodes_to_keep)

In [12]:
for edge in g.edges(data=True):
    g[edge[0]][edge[1]]['weight'] = g[edge[0]][edge[1]]['u']

In [ ]:
netwulf.visualize(g)

In [6]:
print(f"no. nodes: {len(list(g.nodes))} \n no. edges: {len(list(g.edges))}")

no. nodes: 181 
 no. edges: 748


#### wczytanie danych

In [7]:
rides = pd.read_csv("rides.csv", index_col=0)
requests = pd.read_csv("requests.csv", index_col=0)

In [11]:
requests[['pickup_datetime', 'origin', 'destination']]

,pickup_datetime,origin,destination
0,2016-01-02 18:30:02,42440639,42440009
1,2016-01-02 18:30:29,42431106,42438798
2,2016-01-02 18:30:37,42438889,42430347
3,2016-01-02 18:30:47,42428179,42437343
4,2016-01-02 18:58:14,42442895,42437343
...,...,...,...
181,2016-01-02 18:59:15,42437881,42433620
182,2016-01-02 18:59:16,42435477,42435598
183,2016-01-02 18:59:41,42431773,42430308
184,2016-01-02 18:59:45,42445018,42435624


In [14]:
rides[200:][['indexes', 'u_veh', 'PassSecTrav_ns']].rename(columns={'indexes': 'travellers_id', 'u_veh': 'shared_time', 'PassSecTrav_ns':'solo_time'})

,travellers_id,shared_time,solo_time
200,"[54, 100]",1229,1715
201,"[53, 100]",1118,1787
202,"[19, 100]",1055,1817
203,"[66, 100]",1269,1888
204,"[20, 62]",1035,1293
...,...,...,...
987,"[100, 117, 112, 149]",1873,3193
988,"[30, 43, 76, 93]",1501,2526
989,"[30, 43, 62, 76]",1531,2719
990,"[19, 53, 66, 47]",1640,3802


In [15]:
dropped_nodes = [int(k) for k, v in dict(G.degree).items() if v==0]

In [9]:
rides = rides.drop(index=dropped_nodes)
requests = requests.drop(index=dropped_nodes)

print(len(requests))

181


In [10]:
def foo(s):
    return [int(t) for t in s.replace('[', '').replace(']', '').split(',')]


rides['indexes'] = rides.apply(lambda x: foo(x['indexes']), axis=1)

In [11]:
result_no_pooling = sum(requests['ttrav'])
result_no_pooling

110053

In [12]:
def match(im, r, share_discount=0.3, matching_obj="u_veh", plot=False, make_assertion=True, logger=None):
    """
    main call of bipartite matching on a graph
    :param im: possible rides
    :param r: requests
    :param params: parameter (including objective function)
    :param plot:
    :param make_assertion: test the results at the end
    :return: rides, secelcted rides, reuests
    """
    request_indexes = dict()
    request_indexes_inv = dict()
    for i, index in enumerate(r.index.values):
        request_indexes[index] = i
        request_indexes_inv[i] = index

    im_indexes = dict()
    im_indexes_inv = dict()
    for i, index in enumerate(im.index.values):
        im_indexes[index] = i
        im_indexes_inv[i] = index

    im['PassHourTrav_ns'] = im.apply(lambda x: sum([r.loc[_].ttrav for _ in x.indexes]), axis=1)

    r = r.reset_index()

    nR = r.shape[0]

    def add_binary_row(r):
        ret = np.zeros(nR)
        for i in r.indexes:
            ret[request_indexes[i]] = 1
        return ret

    im['row'] = im.apply(add_binary_row, axis=1)  # row to be used as constrain in optimization
    m = np.vstack(im['row'].values).T  # creates a numpy array for the constrains

    im['index'] = im.index.copy()

    im = im.reset_index(drop=True)

    # optimization
    prob = pulp.LpProblem("Matching problem", pulp.LpMinimize)  # problem

    variables = pulp.LpVariable.dicts("r", (i for i in im.index), cat='Binary')  # decision variables

    cost_col = matching_obj
    if cost_col == 'degree':
        costs = im.indexes.apply(lambda x: -(10 ** len(x)))
    elif cost_col == 'u_pax':
        costs = im[cost_col]  # set the costs
    else:
        costs = im[cost_col]  # set the costs

    prob += pulp.lpSum([variables[i] * costs[i] for i in variables]), 'ObjectiveFun'  # ffef

    j = 0  # adding constrains
    for imr in m:
        j += 1
        prob += pulp.lpSum([imr[i] * variables[i] for i in variables if imr[i] > 0]) == 1, 'c' + str(j)

    solver = pulp.get_solver(solver_for_pulp())
    solver.msg = False
    prob.solve(solver)

    assert pulp.value(prob.objective) <= sum(costs[:nR]) + 2  # we did not go above original

    locs = dict()
    for variable in prob.variables():
        i = int(variable.name.split("_")[1])

        locs[im_indexes_inv[i]] = (int(variable.varValue))

    return locs


def solver_for_pulp():
    system = platform.system()
    if system == "Windows":
        return "GLPK_CMD"
    else:
        return "PULP_CBC_CMD"

In [13]:
def calculate_results(rides, requests):
    selected = match(rides, requests)
    fin = rides.loc[rides['selected'] == 1]
    return sum(fin["PassHourTrav_ns"]) - sum(fin["u_veh"])

In [14]:
base_res = calculate_results(rides, requests)
base_res

C:\Users\szmat\.conda\envs\ox\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


38351

In [15]:
def neg_diff_from_base_res(new_res, base_res):
    return new_res - base_res

### Teraz przykładowy podział grafu

In [8]:
g_org = g.copy()

In [27]:
g = g_org.copy()

In [28]:
g = nx.Graph(g)
nx.is_frozen(g)

False

In [29]:
len(g.edges)

748

podział na zbiory

In [30]:
div = nx.community.louvain_communities(g)
div2 = nx.community.greedy_modularity_communities(g, cutoff=3, best_n=3)

usuniecie krawedzi

In [31]:
edge_list = list(g_org.edges)

for edge in edge_list:
    flag = True
    for com in div2:
        if edge[0] in com and edge[1] in com:
            flag = False
    if flag:
        g.remove_edge(edge[0], edge[1])

In [ ]:
netwulf.visualize(g)

In [21]:
len(g.edges)

524

usuniecie ride'ow

In [22]:
rides_org = rides.copy()

In [23]:
len(rides_org)

1729

In [24]:
to_drop = []

for num, r in rides.iterrows():
    flag = True
    for com in div:
        if all([str(t) in com for t in r['indexes']]):
            flag = False
    if flag:
        to_drop.append(num)

rides = rides.drop(index=to_drop)

In [25]:
len(rides)

1190

In [26]:
calculate_results(rides, requests)

C:\Users\szmat\.conda\envs\ox\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


23817

funkcja do maksymalizacji:

In [27]:
neg_diff_from_base_res(calculate_results(rides, requests), base_res)

C:\Users\szmat\.conda\envs\ox\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


-14534